Independent parallel example with Dask, from https://examples.dask.org/applications/embarrassingly-parallel.html
    
Before running this notebook do the following in the terminal of one of the cluster login nodes:

1. Install Miniconda3 as listed in https://www.chpc.utah.edu/documentation/software/python-anaconda.php, that is:
        
> wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  
> bash ./Miniconda3-latest-Linux-x86_64.sh -b -p $HOME/software/pkg/miniconda3  
> mkdir -p $HOME/MyModules/miniconda3  
> cp /uufs/chpc.utah.edu/sys/installdir/python/modules/miniconda3/latest.lua $HOME/MyModules/miniconda3 

2. In the terminal, load the new miniconda3 module and install Dask
> module use $HOME/MyModules  
> module load miniconda3/latest  
> conda install dask "notebook>=6.0"

3. Log into ondemand.chpc.utah.edu with your CHPC creditentials

4. Go to Interactive Apps - Jupyter Notebook on notchpeak

5. In the Environment Setup text box, put:
> module use $HOME/MyModules  
> module load miniconda3/latest  
this will make sure the Jupyter notebook started through the Open OnDemand job will load your own miniconda that has Dask installed.

6. Use notchpeak-shared-short for account and partition, and select your choice of CPU cores and walltime hours (within the listed limits). Then hit Launch to submit the job.

7. Once the job starts, hit the blue Connect to Jupyter button and open this notebook in it.


We are following embarrassingly parallel example at https://examples.dask.org/applications/embarrassingly-parallel.html

In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 3.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35026,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 3.91 GiB
Comm: tcp://127.0.0.1:45634,Total threads: 4
Dashboard: http://127.0.0.1:40061/status,Memory: 3.91 GiB
Nanny: tcp://127.0.0.1:41462,


The Dashboard will not connect in Open OnDemand. There, instead, open a new browser tab and point it to https://ondemand.chpc.utah.edu/rnode/(node_name)/(port)/status<br>
The (node_name) is the same node name that's in the URL of the main Jupyter browser tab (e.g. notch308.ipoib.int.chpc.utah.edu)<br>
The (port) is the Dashboard port (usually 8787).

In [2]:
import time
import random

def costly_simulation(list_param):
    time.sleep(random.random())
    return sum(list_param)

In [3]:
%time costly_simulation([1, 2, 3, 4])

CPU times: user 2.29 ms, sys: 4.12 ms, total: 6.41 ms
Wall time: 70.9 ms


10

In [4]:
import pandas as pd
import numpy as np

input_params = pd.DataFrame(np.random.random(size=(500, 4)),
                            columns=['param_a', 'param_b', 'param_c', 'param_d'])
input_params.head()

,param_a,param_b,param_c,param_d
0,0.019691,0.286815,0.846427,0.681754
1,0.508620,0.804289,0.565490,0.965864
2,0.916745,0.414866,0.438366,0.236070
3,0.661175,0.284014,0.108511,0.936390
4,0.527027,0.528672,0.348630,0.595053


In [5]:
%%time
results = []
for parameters in input_params.values[:10]:
    result = costly_simulation(parameters)
    results.append(result)
results

CPU times: user 179 ms, sys: 38.1 ms, total: 217 ms
Wall time: 5.75 s


[1.8346872833645569,
 2.844263325841346,
 2.0060480760965187,
 1.9900902081661007,
 1.9993809367241955,
 2.0128153909055815,
 2.1571877315576007,
 1.8159885973838386,
 1.6460768037774818,
 3.2859088799725225]

In [6]:
import dask
lazy_results = []

In [7]:
%%time

for parameters in input_params.values[:10]:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)
lazy_results[0]

CPU times: user 5.4 ms, sys: 3.92 ms, total: 9.32 ms
Wall time: 7.15 ms


Delayed('costly_simulation-df74278c-842a-4f66-81e9-f80ec86d23b9')

In [8]:
%time dask.compute(*lazy_results)

CPU times: user 111 ms, sys: 13.7 ms, total: 125 ms
Wall time: 1.72 s


(1.8346872833645569,
 2.844263325841346,
 2.0060480760965187,
 1.9900902081661007,
 1.9993809367241955,
 2.0128153909055815,
 2.1571877315576007,
 1.8159885973838386,
 1.6460768037774818,
 3.2859088799725225)

In [9]:
import dask
lazy_results = []

for parameters in input_params.values:
    lazy_result = dask.delayed(costly_simulation)(parameters)
    lazy_results.append(lazy_result)

futures = dask.persist(*lazy_results)  # trigger computation in the background

In [10]:
%time results = dask.compute(*futures)
results[:5]

CPU times: user 3.14 s, sys: 477 ms, total: 3.62 s
Wall time: 52.4 s


(1.8346872833645569,
 2.844263325841346,
 2.0060480760965187,
 1.9900902081661007,
 1.9993809367241955)

In [11]:
client

Client Scheduler: tcp://127.0.0.1:37257 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 4.19 GB
